# E7 — Altuğ's Ramified Beyond Endoscopy Cancellation at Level $N = pq$

## Goal
Implement the geometric-side cancellation from Altuğ's Beyond Endoscopy program,  
now extended to the ramified (level $N = pq$) setting.

## Background
The Arthur–Selberg trace formula for $\mathrm{GL}_2$ over $\mathbb{Q}$ at level $N$ expands as:
$$\sum_\pi m(\pi) \, h(\pi) = \text{identity} + \text{hyperbolic} + \text{elliptic} + \text{parabolic}$$

The geometric side has $\sim O(N)$ terms, making it computationally hopeless for large $N$.  
Altuğ's insight: after Poisson summation on the hyperbolic terms, massive cancellation occurs,  
leaving only the "cuspidal" contribution plus lower-order residual terms.

The 2025 extension handles the **ramified** case (primes dividing the level),  
which is exactly our $N = pq$ setting.

## Key question
After cancellation, is the remaining cuspidal sum **structurally smaller** or  
**more compressible** than the naive $O(N)$ expansion?

## Method
1. Compute the full geometric side of the trace formula at level $N = pq$.
2. Apply Poisson summation to the hyperbolic orbital integrals.
3. Implement the cancellation (identity + dual terms cancel).
4. Measure the residual: number of terms, magnitude, and structure.
5. Compare naive expansion size vs post-cancellation size.

In [ ]:
import time
import numpy as np
from sage.all import *
import json
import os

## 1. Trace Formula Components for $\Gamma_0(N)$

We use the Eichler–Selberg trace formula for $\mathrm{Tr}(T_n | S_k(\Gamma_0(N)))$.  
For $n = 1$ (the identity Hecke operator), this gives $\dim S_k(\Gamma_0(N))$.  
The four terms are:

- **Identity**: $(k-1)/12 \cdot [\mathrm{SL}_2(\mathbb{Z}) : \Gamma_0(N)]$
- **Elliptic**: sum over elliptic conjugacy classes (quadratic orders)
- **Hyperbolic**: sum over hyperbolic conjugacy classes (traces $|t| > 2$)
- **Parabolic/Eisenstein**: cusp correction terms

In [ ]:
def psi_index(N):
    """Compute [SL_2(Z) : Gamma_0(N)] = N * prod_{p|N} (1 + 1/p)."""
    return N * prod(1 + 1/p for p, _ in factor(N))

def num_cusps_Gamma0(N):
    """Number of cusps of Gamma_0(N) = sum_{d|N} euler_phi(gcd(d, N/d))."""
    return sum(euler_phi(gcd(d, N // d)) for d in divisors(N))

def eichler_selberg_identity_term(N, k=2):
    """Identity contribution to the trace formula."""
    return QQ((k - 1)) / 12 * psi_index(N)

def eichler_selberg_elliptic_term(N, k=2):
    """
    Elliptic contribution for Tr(T_1 | S_k(Gamma_0(N))).
    For n=1, this involves class numbers of imaginary quadratic orders.
    
    For k=2, the elliptic terms come from t with t^2 - 4 < 0,
    i.e., t in {-1, 0, 1} (since |t| < 2).
    Each contributes a sum involving class numbers weighted by
    local representation densities at primes dividing N.
    """
    if k != 2:
        raise NotImplementedError("General k elliptic terms not yet implemented")
    
    total = QQ(0)
    
    # t = 0: discriminant -4, order Z[i]
    # Contribution: -1/2 * h(-4) * local_densities(N, -4)
    # h(-4) = 1, and we need to count solutions to x^2 + 1 ≡ 0 mod N
    rho_4 = _local_embedding_numbers(N, -4)
    total += QQ(-1) / 2 * rho_4
    
    # t = 1: discriminant -3, order Z[ω]
    # h(-3) = 1
    rho_3 = _local_embedding_numbers(N, -3)
    total += QQ(-1) / 2 * rho_3
    
    # t = -1: same discriminant -3 (trace = -1 gives t^2 - 4 = -3)
    total += QQ(-1) / 2 * rho_3
    
    return total

def _local_embedding_numbers(N, D):
    """
    Compute the number of embeddings of the order of discriminant D
    into M_2(Z/NZ) modulo conjugation, which equals
    prod_{p^e || N} (local embedding count at p^e).
    
    For D a fundamental discriminant and N squarefree (our pq case):
    At each prime p | N: count = 1 + kronecker_symbol(D, p).
    """
    result = 1
    for p, e in factor(N):
        if e == 1:
            result *= (1 + kronecker_symbol(D, p))
        else:
            # For higher powers, more complex formula
            # Using Hijikata's formula
            result *= _hijikata_local(p, e, D)
    return result

def _hijikata_local(p, e, D):
    """
    Hijikata's local embedding count for order of discriminant D
    into Eichler order of level p^e.
    For e=1 (our squarefree case): 1 + kronecker(D, p).
    """
    if e == 1:
        return 1 + kronecker_symbol(D, p)
    # General case (not needed for N = pq squarefree)
    chi = kronecker_symbol(D, p)
    if chi == 1:  # split
        return 2 * p^(e-1)
    elif chi == -1:  # inert
        return 0 if e > 1 else 0
    else:  # ramified
        return p^(e-1)

def eichler_selberg_parabolic_term(N, k=2):
    """
    Parabolic (Eisenstein) correction for Tr(T_1 | S_k(Gamma_0(N))).
    For k=2 and n=1: this equals -num_cusps(N)/2.
    """
    if k == 2:
        return QQ(-1) / 2 * num_cusps_Gamma0(N)
    raise NotImplementedError("General k parabolic term")

# Sanity check: sum of terms should give dim S_2(Gamma_0(N))
def trace_formula_dim_check(N, k=2):
    """Verify that the trace formula terms sum to the known dimension."""
    identity = eichler_selberg_identity_term(N, k)
    elliptic = eichler_selberg_elliptic_term(N, k)
    parabolic = eichler_selberg_parabolic_term(N, k)
    
    total = identity + elliptic + parabolic
    expected = dimension_cusp_forms(Gamma0(N), k)
    
    return {
        'N': N,
        'identity': identity,
        'elliptic': elliptic,
        'parabolic': parabolic,
        'total': total,
        'expected': expected,
        'match': total == expected
    }

# Test on a few semiprimes
for N in [15, 77, 143, 323]:
    check = trace_formula_dim_check(N)
    status = "✓" if check['match'] else "✗"
    print(f"N={N:>4}: id={check['identity']:>8}, ell={check['elliptic']:>6}, "
          f"par={check['parabolic']:>6}, total={check['total']:>6}, "
          f"expected={check['expected']:>4} {status}")

## 2. Hyperbolic Orbital Integrals at Level $N = pq$

The hyperbolic contribution to $\mathrm{Tr}(T_n \mid S_k(\Gamma_0(N)))$ sums over  
integer traces $t$ with $t^2 - 4n > 0$ (a perfect square is excluded — those are identity).

For $n = 1$: $t \geq 3$ (and $t \leq -3$ by symmetry), and the sum is:
$$H(N) = \sum_{\substack{t \in \mathbb{Z} \\ t^2 - 4 > 0}} \Phi_\infty(t) \cdot \prod_{p | N} O_p(t, N)$$

where $\Phi_\infty(t)$ is the archimedean orbital integral and  
$O_p(t, N)$ is the local orbital integral at each prime dividing $N$.

For our test function (characteristic function of $K_0(N)$-double coset),  
the local orbital integral at $p | N$ counts lattice points in a  
local conjugacy class.

In [ ]:
def archimedean_orbital_integral(t, k=2):
    """
    Archimedean orbital integral for weight k test function
    at a hyperbolic element with trace t.
    
    For k=2 and the standard test function:
    Phi_inf(t) = sign(t) / |t^2 - 4|^{1/2}  (up to normalizations)
    
    More precisely, for the characteristic function of the
    identity coset: Phi_inf(t) = 1 / (|t^2 - 4|^{1/2}) for |t| > 2.
    """
    disc = t**2 - 4
    if disc <= 0:
        return RR(0)
    return RR(1) / RR(disc).sqrt()

def local_orbital_integral_squarefree(t, p):
    """
    Local orbital integral O_p(t, N) for squarefree level.
    
    For Gamma_0(p) with p prime:
    O_p(t) = number of roots of x^2 - tx + 1 = 0 in Z/pZ
           = number of fixed points of the conjugacy class mod p.
    
    This equals:
    - 2 if t^2 - 4 is a non-zero quadratic residue mod p
    - 1 if t^2 - 4 ≡ 0 mod p
    - 0 if t^2 - 4 is a non-residue mod p
    
    Plus the contribution from the Iwahori level structure.
    For Gamma_0(p): O_p(t) = 1 + kronecker(t^2-4, p) when p ∤ (t^2-4),
    and special value when p | (t^2-4).
    """
    disc = t**2 - 4
    if disc % p == 0:
        # Ramified case for orbital integral
        return 1
    return 1 + kronecker_symbol(disc, p)

def hyperbolic_sum_naive(N, T_max, k=2):
    """
    Compute the hyperbolic contribution by naive summation
    over integer traces |t| >= 3 up to T_max.
    
    Returns the partial sum and the individual terms for analysis.
    """
    primes_of_N = [p for p, _ in factor(N)]
    
    terms = []
    running_sum = RR(0)
    
    for t in range(3, T_max + 1):
        phi_inf = archimedean_orbital_integral(t, k)
        local_product = prod(local_orbital_integral_squarefree(t, p) for p in primes_of_N)
        
        term = phi_inf * local_product
        running_sum += 2 * term  # factor of 2 for ±t symmetry
        
        if local_product > 0:  # only record non-zero terms
            terms.append({
                't': int(t),
                'phi_inf': float(phi_inf),
                'local_product': int(local_product),
                'term': float(term)
            })
    
    return float(running_sum), terms

# Quick test
N_test = 77  # 7 * 11
hyp_sum, hyp_terms = hyperbolic_sum_naive(N_test, T_max=50)
print(f"Hyperbolic sum for N={N_test}, T_max=50: {hyp_sum:.6f}")
print(f"Non-zero terms: {len(hyp_terms)} out of 48 traces")
print(f"First few non-zero terms:")
for term in hyp_terms[:10]:
    print(f"  t={term['t']:>3}: Φ_∞={term['phi_inf']:.4f}, local={term['local_product']}, term={term['term']:.6f}")

## 3. Poisson Summation on Hyperbolic Terms

Altuğ's key insight: apply Poisson summation to the sum over traces $t$.  
The sum $\sum_t f(t)$ becomes $\sum_\xi \hat{f}(\xi)$ where $\hat{f}$ is the Fourier transform.

For $N = pq$ squarefree, we apply Poisson summation mod $N$ to the local orbital integrals.  
The crucial observation is that after this transform:
- The $\xi = 0$ term reproduces the **identity contribution** (plus lower-order).
- The remaining $\xi \neq 0$ terms carry the actual **cuspidal/spectral information**.

The cancellation is: the $\xi = 0$ Poisson term minus the identity term from the geometric side → 0  
(up to lower-order corrections involving $\log$ and powers of $N$).

In [ ]:
def local_orbital_fourier_transform(N, xi):
    """
    Compute the Fourier transform of the local orbital integral
    product at frequency xi mod N.
    
    hat{O}(xi, N) = (1/N) * sum_{t mod N} O(t, N) * e^{2pi i t xi / N}
    
    where O(t, N) = prod_{p | N} O_p(t).
    """
    primes_of_N = [p for p, _ in factor(N)]
    
    total = CC(0)
    zeta_N = CC(exp(2 * pi * I / N))
    
    for t in range(N):
        local_prod = prod(local_orbital_integral_squarefree(t, p) for p in primes_of_N)
        total += local_prod * zeta_N^(t * xi)
    
    return total / N

def poisson_analysis(N, T_max=500):
    """
    Analyze the Poisson-summed hyperbolic terms.
    
    Compute:
    1. The xi=0 Fourier coefficient (should approximate identity term)
    2. The non-zero xi coefficients (carry cuspidal information)
    3. The total power in non-zero modes vs zero mode
    """
    # Compute Fourier transform of local orbital integrals
    fourier_coeffs = {}
    for xi in range(N):
        fourier_coeffs[xi] = local_orbital_fourier_transform(N, xi)
    
    # The xi=0 coefficient = (1/N) * sum_t O(t, N) = average local orbital integral
    zero_mode = fourier_coeffs[0]
    
    # Non-zero modes
    nonzero_magnitudes = [abs(fourier_coeffs[xi]) for xi in range(1, N)]
    
    return {
        'N': int(N),
        'zero_mode': complex(zero_mode),
        'zero_mode_abs': float(abs(zero_mode)),
        'nonzero_magnitudes': [float(m) for m in nonzero_magnitudes],
        'nonzero_mean': float(np.mean(nonzero_magnitudes)) if nonzero_magnitudes else 0,
        'nonzero_max': float(max(nonzero_magnitudes)) if nonzero_magnitudes else 0,
        'nonzero_l2': float(np.sqrt(sum(m**2 for m in nonzero_magnitudes))),
        'num_nonzero_significant': sum(1 for m in nonzero_magnitudes if m > 1e-10),
        'fourier_coeffs': {int(xi): complex(c) for xi, c in fourier_coeffs.items()}
    }

# Test on a small semiprime
pa = poisson_analysis(15)  # 3 * 5
print(f"N = 15 = 3×5:")
print(f"  Zero mode |hat{{O}}(0)|  = {pa['zero_mode_abs']:.6f}")
print(f"  Non-zero: mean = {pa['nonzero_mean']:.6f}, max = {pa['nonzero_max']:.6f}")
print(f"  Non-zero L2 norm = {pa['nonzero_l2']:.6f}")
print(f"  Significant non-zero modes: {pa['num_nonzero_significant']} / {14}")

## 4. Cancellation Measurement

The core test: after Poisson summation, the identity contribution gets  
"absorbed" by the $\xi = 0$ mode. The remaining sum (non-zero $\xi$)  
is the **post-cancellation residual**.

We measure:
- **Term count ratio**: how many non-negligible Fourier modes survive vs $O(N)$ naive terms.
- **Energy ratio**: $L^2$ norm of non-zero modes vs zero mode.
- **Structural sparsity**: are the surviving modes concentrated at special frequencies?

In [ ]:
def cancellation_analysis(N):
    """
    Full cancellation analysis at level N = pq.
    
    Compares the naive geometric expansion against the
    Poisson-transformed version to measure cancellation.
    """
    primes_of_N = [p for p, _ in factor(N)]
    
    # 1. Naive: count non-zero orbital integral values mod N
    naive_nonzero = 0
    naive_total_weight = 0
    for t in range(N):
        local_prod = prod(local_orbital_integral_squarefree(t, p) for p in primes_of_N)
        if local_prod > 0:
            naive_nonzero += 1
            naive_total_weight += local_prod
    
    # 2. Poisson analysis
    pa = poisson_analysis(N)
    
    # 3. Cancellation metrics
    # After removing the xi=0 mode (which captures identity + Eisenstein),
    # how much "signal" remains in the cuspidal (nonzero xi) part?
    
    # Energy concentration ratio
    total_energy = pa['zero_mode_abs']**2 + sum(m**2 for m in pa['nonzero_magnitudes'])
    cuspidal_energy = sum(m**2 for m in pa['nonzero_magnitudes'])
    energy_ratio = cuspidal_energy / total_energy if total_energy > 0 else 0
    
    # Effective rank: how many modes carry significant energy
    threshold = pa['nonzero_max'] * 0.01 if pa['nonzero_max'] > 0 else 1e-10
    effective_rank = sum(1 for m in pa['nonzero_magnitudes'] if m > threshold)
    
    # Check if non-zero modes concentrate at divisor-related frequencies
    divisor_related_modes = []
    for xi in range(1, N):
        # Frequencies related to divisors of N
        if N % xi == 0 or gcd(xi, N) > 1:
            mag = abs(pa['fourier_coeffs'][xi])
            if mag > threshold:
                divisor_related_modes.append((xi, float(mag), gcd(xi, N)))
    
    return {
        'N': int(N),
        'p': int(primes_of_N[0]) if len(primes_of_N) >= 1 else None,
        'q': int(primes_of_N[1]) if len(primes_of_N) >= 2 else None,
        'naive_nonzero_terms': naive_nonzero,
        'naive_total_weight': naive_total_weight,
        'zero_mode_magnitude': pa['zero_mode_abs'],
        'cuspidal_energy_ratio': energy_ratio,
        'effective_rank': effective_rank,
        'total_nonzero_modes': N - 1,
        'significant_nonzero_modes': pa['num_nonzero_significant'],
        'compression_ratio': effective_rank / (N - 1) if N > 1 else 1,
        'divisor_related_modes': divisor_related_modes,
        'poisson_data': pa
    }

# Test on a small semiprime
ca = cancellation_analysis(15)
print(f"N = 15 = 3×5:")
print(f"  Naive non-zero terms: {ca['naive_nonzero_terms']}")
print(f"  Zero mode magnitude: {ca['zero_mode_magnitude']:.6f}")
print(f"  Cuspidal energy ratio: {ca['cuspidal_energy_ratio']:.6f}")
print(f"  Effective rank: {ca['effective_rank']} / {ca['total_nonzero_modes']}")
print(f"  Compression ratio: {ca['compression_ratio']:.4f}")
print(f"  Divisor-related modes: {ca['divisor_related_modes']}")

## 5. Systematic Benchmark Across Semiprimes

In [ ]:
def generate_semiprimes_for_E7(max_N, count=40):
    """Generate semiprimes for E7 analysis. Keep N moderate since
    the Poisson analysis is O(N^2) in the naive implementation."""
    targets = np.logspace(np.log10(15), np.log10(max_N), count)
    semiprimes = []
    seen = set()
    for target in targets:
        N_approx = int(target)
        for offset in range(200):
            candidate = N_approx + offset
            if candidate < 6 or candidate in seen:
                continue
            f = factor(candidate)
            if len(f) == 2 and all(e == 1 for _, e in f):
                p, q = int(f[0][0]), int(f[1][0])
                semiprimes.append((candidate, p, q))
                seen.add(candidate)
                break
    return semiprimes

# Keep max_N moderate for Poisson analysis (O(N^2) cost)
semiprimes_E7 = generate_semiprimes_for_E7(max_N=5000, count=50)
print(f"Generated {len(semiprimes_E7)} semiprimes, range [{semiprimes_E7[0][0]}, {semiprimes_E7[-1][0]}]")

In [ ]:
cancellation_results = []

print(f"{'N':>6} {'p':>5} {'q':>5} {'naive':>6} {'eff_rank':>9} {'compress':>9} {'cusp_E':>8} {'time':>8}")
print("-" * 65)

for N, p, q in semiprimes_E7:
    try:
        t0 = time.perf_counter()
        ca = cancellation_analysis(N)
        elapsed = time.perf_counter() - t0
        
        row = {
            'N': int(N), 'p': int(p), 'q': int(q),
            'naive_terms': ca['naive_nonzero_terms'],
            'effective_rank': ca['effective_rank'],
            'compression_ratio': ca['compression_ratio'],
            'cuspidal_energy_ratio': ca['cuspidal_energy_ratio'],
            'zero_mode_mag': ca['zero_mode_magnitude'],
            'num_divisor_modes': len(ca['divisor_related_modes']),
            'time': elapsed
        }
        cancellation_results.append(row)
        
        print(f"{N:>6} {p:>5} {q:>5} {ca['naive_nonzero_terms']:>6} "
              f"{ca['effective_rank']:>9} {ca['compression_ratio']:>9.4f} "
              f"{ca['cuspidal_energy_ratio']:>8.4f} {elapsed:>8.2f}s")
    except Exception as e:
        print(f"  N={N}: ERROR - {e}")

print(f"\nCompleted {len(cancellation_results)} analyses")

In [ ]:
# Save results
output_path = os.path.join('..', 'data', 'E7_cancellation_results.json')
with open(output_path, 'w') as f:
    json.dump(cancellation_results, f, indent=2)
print(f"Saved {len(cancellation_results)} results to {output_path}")

## 6. Scaling Analysis and Visualization

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy import stats

Ns = np.array([r['N'] for r in cancellation_results])
eff_ranks = np.array([r['effective_rank'] for r in cancellation_results])
compress = np.array([r['compression_ratio'] for r in cancellation_results])
cusp_energy = np.array([r['cuspidal_energy_ratio'] for r in cancellation_results])
naive_terms = np.array([r['naive_terms'] for r in cancellation_results])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Effective rank vs N
ax = axes[0, 0]
ax.scatter(Ns, eff_ranks, s=20, alpha=0.7, color='blue')
ax.plot(Ns, Ns, 'r--', lw=1, alpha=0.5, label='O(N) line')
# Fit power law to effective rank
mask = eff_ranks > 0
if mask.sum() > 2:
    sl, intc, rv, _, se = stats.linregress(np.log10(Ns[mask]), np.log10(eff_ranks[mask]))
    N_fit = np.logspace(np.log10(Ns.min()), np.log10(Ns.max()), 50)
    ax.plot(N_fit, 10**intc * N_fit**sl, 'b--', lw=1.5,
            label=f'Fit: $N^{{{sl:.2f}}}$ (R²={rv**2:.3f})')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Effective rank (post-cancellation)')
ax.set_title('Post-Cancellation Effective Rank')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Compression ratio vs N
ax = axes[0, 1]
ax.scatter(Ns, compress, s=20, alpha=0.7, color='green')
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Compression ratio (eff_rank / (N-1))')
ax.set_title('Compression Ratio vs N')
ax.grid(True, alpha=0.3)

# Plot 3: Cuspidal energy ratio vs N
ax = axes[1, 0]
ax.scatter(Ns, cusp_energy, s=20, alpha=0.7, color='red')
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('Cuspidal energy / Total energy')
ax.set_title('Energy in Cuspidal (non-zero) Modes')
ax.grid(True, alpha=0.3)

# Plot 4: Naive terms vs effective rank
ax = axes[1, 1]
ax.scatter(naive_terms, eff_ranks, s=20, alpha=0.7, color='purple')
ax.plot([0, naive_terms.max()], [0, naive_terms.max()], 'r--', lw=1, label='No compression')
ax.set_xlabel('Naive non-zero terms')
ax.set_ylabel('Effective rank')
ax.set_title('Compression: Naive vs Post-Cancellation')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join('..', 'data', 'E7_cancellation_plots.png'), dpi=150)
plt.show()
print("Saved plots to data/E7_cancellation_plots.png")

## 7. Summary and Verdict

In [ ]:
print("=" * 70)
print("E7 ALTUG BEYOND ENDOSCOPY — SUMMARY")
print("=" * 70)
print()
print(f"Tested {len(cancellation_results)} semiprimes N = pq in [{cancellation_results[0]['N']}, {cancellation_results[-1]['N']}]")
print()

avg_compress = np.mean(compress)
med_compress = np.median(compress)
print(f"Compression ratio: mean = {avg_compress:.4f}, median = {med_compress:.4f}")

avg_cusp_E = np.mean(cusp_energy)
print(f"Cuspidal energy ratio: mean = {avg_cusp_E:.4f}")
print()

# Key scaling result
if mask.sum() > 2:
    print(f"Effective rank scaling: eff_rank ~ N^{sl:.3f} (R²={rv**2:.4f})")
    print()
    if sl < 0.5:
        verdict = "VERY PROMISING — substantial cancellation. Post-cancellation grows sublinearly."
    elif sl < 0.9:
        verdict = "INTERESTING — partial cancellation. Growth reduced but still near-linear."
    elif sl < 1.05:
        verdict = "MARGINAL — nearly linear growth. Cancellation doesn't significantly reduce term count."
    else:
        verdict = "NO SIGNIFICANT CANCELLATION — effective rank grows ≥ N. Beyond Endoscopy doesn't help here."
    print(f"VERDICT: {verdict}")
print()
print("KEY QUESTION: After cancellation, does the cuspidal residual")
print("contain implicit factorization information that's cheaper to extract")
print("than the naive O(N) geometric expansion?")
if avg_compress < 0.5:
    print("=> YES, significant compression observed. Worth investigating further.")
else:
    print("=> Compression is modest. The cuspidal residual may still be too large.")